In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import circuitsvis as cv

import matplotlib.pyplot as plt
import numpy as np
import re
from collections import OrderedDict
import os
import random
import json
import string

from transformers import GPT2LMHeadModel, AutoConfig, PreTrainedTokenizerBase
from transformer_lens import HookedTransformer, HookedTransformerConfig
import importlib
from utils import*

torch.set_grad_enabled(False)
torch.set_printoptions(sci_mode=False)


/Users/huangxt/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/huangxt/anaconda3/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /Users/huangxt/anaconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <CFB4297E-118A-3D27-BF39-55106327812A> /Users/huangxt/anaconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/huangxt/anaconda3/lib/python3.11/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta.

In [2]:
probed_model_path = "t3_model"

probed_model = GPT2LMHeadModel.from_pretrained(probed_model_path)
tokenizer = customTokenizer([str(i) for i in range(150)])

hooked_model = HookedTransformer.from_pretrained(
        "gpt2",
        hf_model=probed_model,
        tokenizer=None,
        n_embd=probed_model.config.n_embd,
        n_layer=probed_model.config.n_layer,
        n_head=probed_model.config.n_head,
        vocab_size=probed_model.config.vocab_size,
        n_positions=probed_model.config.n_positions,
        n_ctx=probed_model.config.n_positions,
)

hooked_model.tokenizer = tokenizer

dataset = iter(UniqueCopyDataset(tokenizer, (50, 60), -1, 3))


Loaded pretrained model gpt2 into HookedTransformer


In [3]:
x, _, _ = next(dataset)


pred, cache = hooked_model.run_with_cache(torch.LongTensor([x]), return_type="logits")

s = x.index(tokenizer.sep_token_id)
e = x.index(tokenizer.eos_token_id)
pred = pred.squeeze(0)[s:e].argmax(dim=1).tolist()

print("input", " ".join(tokenizer.convert_ids_to_tokens(x)))
print("pred", " ".join(tokenizer.convert_ids_to_tokens(pred)))



input <bos> 27 138 56 140 110 123 50 141 59 8 39 74 70 142 139 60 108 115 52 1 112 82 116 90 68 63 24 19 135 125 129 101 29 133 132 65 128 4 111 23 57 104 9 136 64 28 85 97 93 130 53 149 95 <sep> 56 123 59 74 139 115 112 90 24 125 29 65 111 104 64 97 53 <eos>
pred 56 123 59 74 139 115 112 90 24 125 29 65 111 104 64 97 53 <eos>


In [4]:
layer_idx = 0
attention_pattern = cache[f"blocks.{layer_idx}.attn.hook_pattern"][0]

# attention_pattern[0].diagonal(-1)

weighted_by_v_norm = False
if weighted_by_v_norm:
    v_vector = cache[f"blocks.{layer_idx}.attn.hook_v"][0]
    v_norm = torch.linalg.vector_norm(v_vector, dim=-1)
    attention_pattern = attention_pattern * v_norm.transpose(0, 1).unsqueeze(1)
    attention_pattern /= 3
str_tokens = ["&nbsp;"+i.strip("<>")+"&nbsp;" for i in tokenizer.convert_ids_to_tokens(x)]
cv.attention.attention_patterns(tokens=str_tokens, attention=attention_pattern)